In [1]:
import torch
import torch.nn.functional as F
import numpy as np

# ------------------------------------------------------------------
# 1) “Image”  (4 × 4)   and   Kernel  (3 × 3)
v_img = torch.tensor([[1,  2,  3,  4],
                      [5,  6,  7,  8],
                      [9, 10, 11, 12],
                      [13,14, 15, 16]], dtype=torch.float32)

kernel = torch.tensor([[1, 2, 3],
                       [4, 5, 6],
                       [7, 8, 9]], dtype=torch.float32)

# Reshape to NCHW and OIHW so F.conv2d is happy
v_bchw = v_img.unsqueeze(0).unsqueeze(0)      # (1,1,4,4)
w_oihw = kernel.unsqueeze(0).unsqueeze(0)     # (1,1,3,3)

y_torch = F.conv2d(v_bchw, w_oihw, stride=1, padding=0)  # (1,1,2,2)
print("PyTorch output:\n", y_torch.squeeze())            # [[348, 393], [528, 573]]

# ------------------------------------------------------------------
# 2) Build the explicit convolution matrix A  (4 × 16)
H, W = 4, 4
out_h, out_w = 2, 2
A = torch.zeros(out_h * out_w, H * W)

for p in range(H * W):
    # make a delta-image with a 1.0 at pixel p
    delta = torch.zeros_like(v_bchw)
    r, c = divmod(p, W)
    delta[0, 0, r, c] = 1.0

    # convolve: the result is this *column* of A
    A[:, p] = F.conv2d(delta, w_oihw).flatten()

# Sanity check:   A @ vec(v)  ==  y_torch ?
v_flat = v_img.flatten()
y_from_A = A @ v_flat
assert torch.allclose(y_from_A, y_torch.flatten())

print("\nConvolution matrix A (rows = outputs, cols = input pixels):\n")
print(A)

PyTorch output:
 tensor([[348., 393.],
        [528., 573.]])

Convolution matrix A (rows = outputs, cols = input pixels):

tensor([[1., 2., 3., 0., 4., 5., 6., 0., 7., 8., 9., 0., 0., 0., 0., 0.],
        [0., 1., 2., 3., 0., 4., 5., 6., 0., 7., 8., 9., 0., 0., 0., 0.],
        [0., 0., 0., 0., 1., 2., 3., 0., 4., 5., 6., 0., 7., 8., 9., 0.],
        [0., 0., 0., 0., 0., 1., 2., 3., 0., 4., 5., 6., 0., 7., 8., 9.]])
